<a href="https://colab.research.google.com/github/MicaelaRomeroC3/sotto/blob/main/Sottotempo_Registro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Evaluaciones prueba Decathlon.xlsx'

# Cargar el archivo Excel
df = pd.read_excel(file_path)

# Mostrar las primeras filas del archivo
df.head()


,ID,IDEVENTCLIENTE,EQUIPO,CIP,ID SERVICIO,SERVICIO,EMPRESA,DISPOSITIVO,MEDIDOR,FECHA EVALUACIÓN,...,C13. CALIDAD DE LA RESPUESTA.COND.LA RESPUESTA PROPORCIONADA DURANTE LA CONVERSACIÓN DE WHATSAPP ES FALSA. MUESTRA NULO CONOCIMIENTO (ENTENDIMIENTO) DE LA CONSULTA O PRODUCTO.,C13. CALIDAD DE LA RESPUESTA.COND.EL CHAT DE WHATSAPP NO HA SIDO ATENDIDO TRAS PASARSE EL PLAZO DE 5 MINUTOS Y NO RECIBIR RESPUESTA POR PARTE DE UN ASESOR.,C14. FUNCIONALIDAD.COND.EL LINK FACILITADO POR EL ASESOR LLEVA A UNA PÁGINA CON LA SOLUCIÓN A LA CONSULTA PLANTEADA POR EL CLIENTE.,C14. FUNCIONALIDAD.COND.EL LINK FACILITADO POR EL ASESOR LLEVA A UNA PÁGINA DIFERENTE A LO CONSULTA DESCRITA POR EL CLIENTE.,C15. GESTIÓN CORRECTA DE LA SOLICITUD,C15. GESTIÓN CORRECTA DE LA SOLICITUD.COND.EL ASESOR SÍ DERIVA AL CANAL CORRECTO DONDE EL CLIENTE PUEDE SOLVENTAR LA CONSULTA.,C15. GESTIÓN CORRECTA DE LA SOLICITUD.COND.EL ASESOR NO DERIVA AL CANAL CORRECTO DONDE EL CLIENTE PUEDE SOLVENTAR LA CONSULTA.,C16. SOLUCIÓN AL PRIMER CONTACTO (FCR),C16. SOLUCIÓN AL PRIMER CONTACTO (FCR).COND.SE SOLUCIONA LA CONSULTA EN EL PRIMER CONTACTO Y NO ES NECESARIO REALIZAR UN CONTACTO POSTERIOR.,C16. SOLUCIÓN AL PRIMER CONTACTO (FCR).COND.NO SE SOLUCIONA LA CONSULTA EN EL PRIMER CONTACTO Y ES NECESARIO REALIZAR UN CONTACTO POSTERIOR.
0,101S_969-OP_11_291_20250226H_12_03,LLAMADA PRUEBA 1,Decathlon 100,,969,Decathlon Teléfono,Decathlon,,10057,26/02/2025,...,,,,,,,,,,
1,101S_969-OP_11_291_20250226H_12_07,LLAMADA PRUEBA 2,Decathlon 100,,969,Decathlon Teléfono,Decathlon,,10057,26/02/2025,...,,,,,,,,,,
2,101S_969-OP_11_291_20250226H_12_11,LLAMADA PRUEBA 3,Decathlon 100,,969,Decathlon Teléfono,Decathlon,,10057,26/02/2025,...,,,,,,,,,,
3,101S_969-OP_11_291_20250226H_12_15,LLAMADA PRUEBA 4,Decathlon 100,,969,Decathlon Teléfono,Decathlon,,10057,26/02/2025,...,,,,,,,,,,
4,101S_969-OP_11_291_20250226H_12_18,LLAMADA PRUEBA 5,Decathlon 100,,969,Decathlon Teléfono,Decathlon,,10057,26/02/2025,...,,,,,,,,,,


# **Extraer el canal de la columna SERVICIO**

In [20]:
# Lista de palabras clave
servicios_validos = ['Teléfono', 'Mail/Formulario', 'Web', 'Twitter', 'Facebook', 'Chat Web', 'Whatsapp']

# Función para extraer la palabra clave válida
def limpiar_servicio(servicio):
    for palabra in servicios_validos:
        if palabra in servicio:
            return palabra
    return None  # Retorna None si no encuentra ninguna palabra válida

# Aplicar la función a la columna "Servicio"
df['Servicio Limpio'] = df['SERVICIO'].apply(limpiar_servicio)

# Mostrar el DataFrame con la nueva columna
print(df['Servicio Limpio'])

0            Teléfono
1            Teléfono
2            Teléfono
3            Teléfono
4            Teléfono
5            Teléfono
6     Mail/Formulario
7     Mail/Formulario
8     Mail/Formulario
9     Mail/Formulario
10    Mail/Formulario
11    Mail/Formulario
12                Web
13                Web
14                Web
15                Web
16                Web
17                Web
18            Twitter
19            Twitter
20            Twitter
21           Facebook
22           Facebook
23           Facebook
24                Web
25                Web
26                Web
27           Whatsapp
28           Whatsapp
29           Whatsapp
Name: Servicio Limpio, dtype: object


**Solo si se requiere descargar ejecutar la siguiente casilla**

In [21]:
# Guardar el DataFrame modificado como archivo Excel
df.to_excel('Servicio-Limpio.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Servicio-Limpio.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Crear hoja REGISTRO**

In [27]:
# Renombramos columnas para facilitar el trabajo
df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"), inplace=True)

# Mapear los canales a tipo contacto
def map_tipo_contacto(canal):
    canal = canal.lower()
    if canal in ['facebook', 'twitter']:
        return 'Redes Sociales'
    elif canal in ['chat web', 'whatsapp']:
        return 'Chat'
    else:
        return canal

# Aplicamos el mapeo
df['tipo_contacto'] = df['servicio_limpio'].apply(map_tipo_contacto)

# Capitalizamos la primera letra de cada tipo_contacto
df['tipo_contacto'] = df['tipo_contacto'].str.capitalize()

# Formateamos la fecha
df['fecha'] = pd.to_datetime(df['fecha_evaluación']).dt.date

# Columna fija
df['tipo_de_acción'] = 'evaluada'

# Agrupamos por fecha, empresa y tipo_contacto (canal)
df = df.groupby(['fecha', 'empresa', 'tipo_contacto']).size().reset_index(name='cantidad')

# Añadimos columna de tipo_de_acción (ya está aplicada arriba, pero por si acaso)
df['tipo_de_acción'] = 'Evaluada'

# Reordenamos columnas
df = df[['fecha', 'empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']]

# Mostramos resultado
df.head()

<ipython-input-27-ba3dfa087adc>:21: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['fecha'] = pd.to_datetime(df['fecha_evaluación']).dt.date


,fecha,empresa,tipo_contacto,tipo_de_acción,cantidad
0,2025-02-26,Decathlon,Chat,Evaluada,3
1,2025-02-26,Decathlon,Mail/formulario,Evaluada,6
2,2025-02-26,Decathlon,Redes sociales,Evaluada,6
3,2025-02-26,Decathlon,Teléfono,Evaluada,6
4,2025-02-26,Decathlon,Web,Evaluada,9


In [31]:
print(df.columns.tolist())


['fecha', 'empresa', 'tipo_contacto', 'tipo_de_acción', 'cantidad']


In [23]:
# Guardar el DataFrame modificado como archivo Excel
df.to_excel('Registro.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Registro.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Crear hoja Recuento por TIPO DE CONTACTO**

In [40]:
import pandas as pd

# Ruta del archivo Excel en tu Google Drive
file_path = '/content/drive/MyDrive/Sottotempo/Crear - Recuento por TIPO DE CONTACTO/Resumen Planificación.xlsx'

# Cargar el archivo Excel
df2 = pd.read_excel(file_path)

# Mostrar las primeras filas del archivo
df2.head()

,Empresa,Categoría,Inicio,Fin,BBDD,Observaciones BBDD,Compra,Fecha Informe,Teléfono,Mail/formulario,...,Twitter,Facebook,Chat,WhatsApp,Confirmación,Informe,Alta en eAlicia,Escenarios C3,Emisión (horas),Evaluación (horas)
0,SECTOR ALARMA,Sistemas de Seguridad,2025-05-05,2025-06-27,NaN,NaN,NaN,NaN,135,55,...,0,0,0,0,NaN,NaN,NaN,NaN,17.250000,25.875
1,ADT,Sistemas de Seguridad,2025-05-05,2025-06-27,NaN,NaN,NaN,NaN,45,18,...,0,0,0,0,No,NaN,NaN,NaN,5.716667,8.575
2,Securitas,Sistemas de Seguridad,2025-05-05,2025-06-27,NaN,NaN,NaN,NaN,45,18,...,0,0,0,0,NaN,NaN,NaN,NaN,5.716667,8.575
3,Prosegur,Sistemas de Seguridad,2025-05-05,2025-06-27,NaN,NaN,NaN,NaN,45,19,...,0,0,0,0,NaN,NaN,NaN,NaN,5.816667,8.725
4,Bruneau,Material de Oficina,2025-05-05,2025-06-27,NaN,NaN,NaN,NaN,135,55,...,0,0,12,0,NaN,NaN,NaN,NaN,18.650000,27.975


**Unir los dos excel, obteniendo evaluada, realizada y total**

In [41]:
# ------------------------
# Paso 1: Pivotear df (acciones) → Evaluada y Realizada por canal
# ------------------------
# Hacemos el pivot
df_pivot = df.pivot_table(index=['empresa', 'tipo_contacto'],
                          columns='tipo_de_acción',
                          values='cantidad',
                          aggfunc='sum',
                          fill_value=0)

# Aseguramos que ambas columnas existan
df_pivot = df_pivot.reindex(columns=['Evaluada', 'Realizada'], fill_value=0)

# Volvemos a poner el índice como columnas normales
df_pivot = df_pivot.reset_index()
df_pivot.columns.name = None
df_pivot.rename(columns={'empresa': 'Empresa'}, inplace=True)

# ------------------------
# Paso 2: Convertir df2 (interacciones) a formato largo (canal en filas)
# ------------------------
canales = ['Teléfono', 'Mail/formulario', 'Web', 'Facebook', 'Twitter', 'Chat', 'WhatsApp']

df2_melt = df2.melt(id_vars=['Empresa', 'Inicio', 'Fin'],
                    value_vars=canales,
                    var_name='tipo_contacto',
                    value_name='Total_temp')  # evitamos conflicto

# Renombrar después si querés mantener "Total"
df2_melt.rename(columns={'Total_temp': 'Total'}, inplace=True)

# ------------------------
# Paso 3: Unir df2_melt con df_pivot
# ------------------------
df_merged = pd.merge(df2_melt, df_pivot, on=['Empresa', 'tipo_contacto'], how='left')

# Si no hay datos en df (Evaluada o Realizada), rellenamos con 0
df_merged[['Evaluada', 'Realizada']] = df_merged[['Evaluada', 'Realizada']].fillna(0)

# ------------------------
# Paso 4: Pivotear final → una columna por canal con sufijos
# ------------------------
df_final = df_merged.pivot_table(index=['Empresa', 'Inicio', 'Fin'],
                                 columns='tipo_contacto',
                                 values=['Evaluada', 'Realizada', 'Total'],
                                 fill_value=0)

# Aplanar columnas: ('Evaluada', 'Facebook') → Facebook_Evaluada
df_final.columns = [f"{canal}_{tipo}" for tipo, canal in df_final.columns]
df_final = df_final.reset_index()

# ------------------------
# Mostrar resultado
# ------------------------
df_final.head()

,Empresa,Inicio,Fin,Chat_Evaluada,Facebook_Evaluada,Mail/formulario_Evaluada,Teléfono_Evaluada,Twitter_Evaluada,Web_Evaluada,WhatsApp_Evaluada,...,Twitter_Realizada,Web_Realizada,WhatsApp_Realizada,Chat_Total,Facebook_Total,Mail/formulario_Total,Teléfono_Total,Twitter_Total,Web_Total,WhatsApp_Total
0,ADT,2025-05-05,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,18.0,45.0,0.0,2.0,0.0
1,Acer,2025-05-05,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,12.0,6.0,55.0,135.0,6.0,6.0,0.0
2,Adeslas,2025-05-05,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,18.0,45.0,0.0,2.0,0.0
3,Aeda Homes,2025-05-05,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,27.0,68.0,0.0,3.0,0.0
4,Aegon Salud,2025-05-05,2025-06-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,55.0,135.0,0.0,6.0,0.0


**Ordenar columnas para que esten juntos por canal**

In [46]:
# Paso final: ordenar columnas agrupadas por canal
orden_final = ['Empresa', 'Inicio', 'Fin']
tipos = ['Evaluada', 'Realizada', 'Total']
for canal in canales:
    for tipo in tipos:
        col = f"{canal}_{tipo}"
        if col in df_final.columns:
            orden_final.append(col)

# Reordenar el DataFrame
df_final = df_final[orden_final]

df_final.head()

,Empresa,Inicio,Fin,Teléfono_Evaluada,Teléfono_Realizada,Teléfono_Total,Mail/formulario_Evaluada,Mail/formulario_Realizada,Mail/formulario_Total,Web_Evaluada,...,Facebook_Total,Twitter_Evaluada,Twitter_Realizada,Twitter_Total,Chat_Evaluada,Chat_Realizada,Chat_Total,WhatsApp_Evaluada,WhatsApp_Realizada,WhatsApp_Total
0,ADT,2025-05-05,2025-06-27,0.0,0.0,45.0,0.0,0.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Acer,2025-05-05,2025-06-27,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,6.0,0.0,0.0,6.0,0.0,0.0,12.0,0.0,0.0,0.0
2,Adeslas,2025-05-05,2025-06-27,0.0,0.0,45.0,0.0,0.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aeda Homes,2025-05-05,2025-06-27,0.0,0.0,68.0,0.0,0.0,27.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aegon Salud,2025-05-05,2025-06-27,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Obtener Total_general, total_evaluadas y total_realizadas**

In [55]:
# Identificamos columnas por tipo
cols_total = [col for col in df_final.columns if col.endswith('_Total')]
cols_evaluada = [col for col in df_final.columns if col.endswith('_Evaluada')]
cols_realizada = [col for col in df_final.columns if col.endswith('_Realizada')]

# Creamos las columnas de suma total
df_final['Total_general'] = df_final[cols_total].sum(axis=1)
df_final['Total_evaluadas'] = df_final[cols_evaluada].sum(axis=1)
df_final['Total_realizadas'] = df_final[cols_realizada].sum(axis=1)

df_final.head()


,Empresa,Inicio,Fin,Teléfono_Evaluada,Teléfono_Realizada,Teléfono_Total,Mail/formulario_Evaluada,Mail/formulario_Realizada,Mail/formulario_Total,Web_Evaluada,...,WhatsApp_Realizada,WhatsApp_Total,Total_general,Total_evaluadas,Total_realizadas,Dias_laborables_restantes,Por_evaluar,Por_realizar,A_realizar_por_dia,A_evaluar_por_dia
0,ADT,2025-05-05,2025-06-27,0.0,0.0,45.0,0.0,0.0,18.0,0.0,...,0.0,0.0,65.0,0.0,0.0,39,65.0,65.0,1.666667,1.666667
1,Acer,2025-05-05,2025-06-27,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,220.0,0.0,0.0,39,220.0,220.0,5.641026,5.641026
2,Adeslas,2025-05-05,2025-06-27,0.0,0.0,45.0,0.0,0.0,18.0,0.0,...,0.0,0.0,65.0,0.0,0.0,39,65.0,65.0,1.666667,1.666667
3,Aeda Homes,2025-05-05,2025-06-27,0.0,0.0,68.0,0.0,0.0,27.0,0.0,...,0.0,0.0,98.0,0.0,0.0,39,98.0,98.0,2.512821,2.512821
4,Aegon Salud,2025-05-05,2025-06-27,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,196.0,0.0,0.0,39,196.0,196.0,5.025641,5.025641


**Obtener los dias laborables restantes**

In [54]:
import numpy as np
from datetime import datetime

# Asegurar que la columna Fin está en formato datetime
df_final['Fin'] = pd.to_datetime(df_final['Fin'])

# Fecha de hoy
hoy = np.datetime64(datetime.today().date(), 'D')

# Convertir 'Fin' a datetime64[D] para que funcione con busday_count
fechas_fin = df_final['Fin'].values.astype('datetime64[D]')

# Calcular días hábiles restantes
df_final['Dias_laborables_restantes'] = np.busday_count(hoy, fechas_fin)

df_final.head()

,Empresa,Inicio,Fin,Teléfono_Evaluada,Teléfono_Realizada,Teléfono_Total,Mail/formulario_Evaluada,Mail/formulario_Realizada,Mail/formulario_Total,Web_Evaluada,...,WhatsApp_Realizada,WhatsApp_Total,Total_general,Total_evaluadas,Total_realizadas,Dias_laborables_restantes,Por_evaluar,Por_realizar,A_realizar_por_dia,A_evaluar_por_dia
0,ADT,2025-05-05,2025-06-27,0.0,0.0,45.0,0.0,0.0,18.0,0.0,...,0.0,0.0,65.0,0.0,0.0,39,65.0,65.0,1.666667,1.666667
1,Acer,2025-05-05,2025-06-27,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,220.0,0.0,0.0,39,220.0,220.0,5.641026,5.641026
2,Adeslas,2025-05-05,2025-06-27,0.0,0.0,45.0,0.0,0.0,18.0,0.0,...,0.0,0.0,65.0,0.0,0.0,39,65.0,65.0,1.666667,1.666667
3,Aeda Homes,2025-05-05,2025-06-27,0.0,0.0,68.0,0.0,0.0,27.0,0.0,...,0.0,0.0,98.0,0.0,0.0,39,98.0,98.0,2.512821,2.512821
4,Aegon Salud,2025-05-05,2025-06-27,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,196.0,0.0,0.0,39,196.0,196.0,5.025641,5.025641


**Obtener columnas por evaluar y por realizar según el total general**

In [53]:
df_final['Por_evaluar'] = df_final['Total_general'] - df_final['Total_evaluadas']
df_final['Por_realizar'] = df_final['Total_general'] - df_final['Total_realizadas']

df_final.head()

,Empresa,Inicio,Fin,Teléfono_Evaluada,Teléfono_Realizada,Teléfono_Total,Mail/formulario_Evaluada,Mail/formulario_Realizada,Mail/formulario_Total,Web_Evaluada,...,WhatsApp_Realizada,WhatsApp_Total,Total_general,Total_evaluadas,Total_realizadas,Dias_laborables_restantes,Por_evaluar,Por_realizar,A_realizar_por_dia,A_evaluar_por_dia
0,ADT,2025-05-05,2025-06-27,0.0,0.0,45.0,0.0,0.0,18.0,0.0,...,0.0,0.0,65.0,0.0,0.0,39,65.0,65.0,1.666667,1.666667
1,Acer,2025-05-05,2025-06-27,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,220.0,0.0,0.0,39,220.0,220.0,5.641026,5.641026
2,Adeslas,2025-05-05,2025-06-27,0.0,0.0,45.0,0.0,0.0,18.0,0.0,...,0.0,0.0,65.0,0.0,0.0,39,65.0,65.0,1.666667,1.666667
3,Aeda Homes,2025-05-05,2025-06-27,0.0,0.0,68.0,0.0,0.0,27.0,0.0,...,0.0,0.0,98.0,0.0,0.0,39,98.0,98.0,2.512821,2.512821
4,Aegon Salud,2025-05-05,2025-06-27,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,196.0,0.0,0.0,39,196.0,196.0,5.025641,5.025641


**Calcular la cantidad de a realizar y evaluar según los días laborables restantes**

In [57]:
# Asegurarse de evitar división por cero o días negativos
df_final['Dias_laborables_restantes'] = df_final['Dias_laborables_restantes'].clip(lower=1)

# Calcular valores por día
df_final['A_realizar_por_dia'] = ((df_final['Total_general'] - df_final['Total_realizadas']) / df_final['Dias_laborables_restantes']).round(0).astype(int)
df_final['A_evaluar_por_dia'] = ((df_final['Total_general'] - df_final['Total_evaluadas']) / df_final['Dias_laborables_restantes']).round(0).astype(int)

df_final

,Empresa,Inicio,Fin,Teléfono_Evaluada,Teléfono_Realizada,Teléfono_Total,Mail/formulario_Evaluada,Mail/formulario_Realizada,Mail/formulario_Total,Web_Evaluada,...,WhatsApp_Realizada,WhatsApp_Total,Total_general,Total_evaluadas,Total_realizadas,Dias_laborables_restantes,Por_evaluar,Por_realizar,A_realizar_por_dia,A_evaluar_por_dia
0,ADT,2025-05-05,2025-06-27,0.0,0.0,45.0,0.0,0.0,18.0,0.0,...,0.0,0.0,65.0,0.0,0.0,39,65.0,65.0,2,2
1,Acer,2025-05-05,2025-06-27,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,220.0,0.0,0.0,39,220.0,220.0,6,6
2,Adeslas,2025-05-05,2025-06-27,0.0,0.0,45.0,0.0,0.0,18.0,0.0,...,0.0,0.0,65.0,0.0,0.0,39,65.0,65.0,2,2
3,Aeda Homes,2025-05-05,2025-06-27,0.0,0.0,68.0,0.0,0.0,27.0,0.0,...,0.0,0.0,98.0,0.0,0.0,39,98.0,98.0,3,3
4,Aegon Salud,2025-05-05,2025-06-27,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,196.0,0.0,0.0,39,196.0,196.0,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Up Spain,2025-05-05,2025-06-27,0.0,0.0,67.0,0.0,0.0,28.0,0.0,...,0.0,0.0,98.0,0.0,0.0,39,98.0,98.0,3,3
81,Viatris,2025-05-05,2025-06-27,0.0,0.0,135.0,0.0,0.0,55.0,0.0,...,0.0,0.0,196.0,0.0,0.0,39,196.0,196.0,5,5
82,Whirlpool,2025-05-05,2025-06-27,0.0,0.0,34.0,0.0,0.0,14.0,0.0,...,0.0,0.0,50.0,0.0,0.0,39,50.0,50.0,1,1
83,Xiaomi,2025-05-05,2025-06-27,0.0,0.0,45.0,0.0,0.0,19.0,0.0,...,0.0,0.0,66.0,0.0,0.0,39,66.0,66.0,2,2


**Extraer el archivo final**

In [58]:
# Guardar el DataFrame modificado como archivo Excel
df_final.to_excel('Recuento por TIPO DE CONTACTO.xlsx', index=False)

# Descargar el archivo a tu computadora
from google.colab import files
files.download('Recuento por TIPO DE CONTACTO.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>